In [3]:
import os
import h2o
import pandas as pd
import numpy as np
import datetime as dt
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ONaiveBayesEstimator

In [4]:
import matplotlib.pylab as plt
%matplotlib inline

In [128]:
h2o.shutdown()

[WARNING] in <ipython-input-128-bd1e984f485f> line 1:
    >>> h2o.shutdown()
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
H2O session _sid_bf2f closed.


In [5]:
h2o.init(nthreads = -1, max_mem_size = 25)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,42 mins 12 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_laith_g68owb
H2O cluster total nodes:,1
H2O cluster free memory:,21.26 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [35]:
def prepare_data(with_validation = True):
    train = h2o.import_file(os.path.realpath("../data/train2.csv"))
    test  = h2o.import_file(os.path.realpath("../data/test2.csv"))
    
    x = train.columns
    y = "target"
    x.remove(y)
    
    test_id = test['id'].as_data_frame(True).id.values
    
    features_bin = [i for i in x if set(i)&set('bin')==set('bin')]
    features_cat = [i for i in x if (set(i)&set('cat')==set('cat'))&(set(i)&set('avg')!=set('avg'))]
    features_con = list(set(x) - (set(features_bin)|set(features_cat)))
    
    train[features_cat] = train[features_cat].asfactor()
    train[features_bin] = train[features_bin].asfactor()
    train[features_con] = train[features_con].asnumeric()
    
    test[features_cat] = test[features_cat].asfactor()
    test[features_bin] = test[features_bin].asfactor()
    test[features_con] = test[features_con].asnumeric()
    
    train['target'] = train['target'].asfactor()
    
    return train, test, x, y, test_id

In [36]:
train, test, x, y, test_id = prepare_data()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [37]:
nfolds = 5

naivebayes Model Build progress: |████████████████████████████████████████| 100%


In [38]:
gscore = 0
selected_features = []
for i in range(100):
    feature_numbers = np.random.choice(range(15,50), size=1)
    random_features= list(np.random.choice(x,feature_numbers))
    print(feature_numbers)
    my_nbe = H2ONaiveBayesEstimator(laplace=0,
                                    compute_metrics=True,
                                    nfolds=nfolds,
                                    fold_assignment="Stratified",
                                    keep_cross_validation_predictions=True,
                                    seed=1)
    my_nbe.train(x=random_features, y=y, training_frame=train)
    temp_gscore = float(my_nbe.cross_validation_metrics_summary().as_data_frame().iloc[1]['mean'])
    if temp_gscore>gscore:
        print(temp_gscore)
        gscore=temp_gscore
        selected_features = random_features

[40]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
0.5952763
[21]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[29]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
0.5975518
[33]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[46]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[17]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
0.6087951
[30]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[40]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[33]
naivebayes Model Build progress: |█████████████████████████████████████████████| 100%
[28]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[44]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
0.6128357
[31]
naiv

naivebayes Model Build progress: |████████████████████████████████████████| 100%
[39]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[15]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[41]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[35]
naivebayes Model Build progress: |████████████████████████████████████████| 100%
[19]
naivebayes Model Build progress: |████████████████████████████████████████| 100%


In [40]:
my_nbe = H2ONaiveBayesEstimator(laplace=0,
                                compute_metrics=True,
                                nfolds=nfolds,
                                fold_assignment="Stratified",
                                keep_cross_validation_predictions=True,
                                seed=1)
my_nbe.train(x=selected_features, y=y, training_frame=train)

naivebayes Model Build progress: |████████████████████████████████████████| 100%


In [41]:
my_nbe.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.7839491,0.015431979,0.81650805,0.7658846,0.79217637,0.7542958,0.79088044
1,auc,0.6147174,0.0038015617,0.62210876,0.6181887,0.6113995,0.6065651,0.6153251
2,err,0.21605095,0.015431979,0.18349196,0.2341154,0.20782365,0.24570419,0.20911954
3,err_count,25728.0,1901.7823,21766.0,27830.0,24720.0,29499.0,24825.0
4,f0point5,0.07095106,0.0020541463,0.075263694,0.06844189,0.070581794,0.06739358,0.073074356
5,f1,0.100387126,0.0018699407,0.10398485,0.09836066,0.09958476,0.097089164,0.1029162
6,f2,0.17189573,0.001931489,0.16815338,0.17475192,0.16904926,0.17356855,0.17395553
7,lift_top_group,2.3796537,0.12213519,2.237822,2.482129,2.6340551,2.3960474,2.1482153
8,logloss,0.29211006,0.0036475854,0.28854528,0.2842997,0.29404783,0.29917955,0.29447794
9,max_per_class_error,0.6696354,0.021283314,0.71431804,0.63762236,0.6840037,0.634478,0.6777552


In [39]:
selected_features

['ps_car_13',
 'ps_car_03_cat_avg',
 'ps_car_08_cat',
 'ps_reg_01_plus_ps_car_02_cat',
 'ps_car_06_cat',
 'ps_ind_12_bin',
 'ps_car_06_cat_avg',
 'ps_ind_08_bin',
 'ps_reg_01',
 'ps_reg_01_plus_ps_car_02_cat_avg',
 'ps_ind_05_cat_avg',
 'ps_car_15',
 'ps_ind_02_cat_avg',
 'ps_car_07_cat',
 'ps_car_01_cat',
 'ps_car_04_cat',
 'ps_car_08_cat_avg',
 'ps_reg_01_plus_ps_car_04_cat',
 'ps_car_06_cat',
 'ps_reg_01_plus_ps_car_02_cat',
 'ps_ind_01',
 'ps_car_03_cat',
 'ps_ind_18_bin',
 'ps_ind_02_cat',
 'ps_car_07_cat_avg',
 'ps_ind_05_cat_avg',
 'ps_car_05_cat',
 'ps_car_04_cat',
 'ps_car_06_cat_avg',
 'ps_car_07_cat_avg',
 'ps_ind_08_bin',
 'ps_car_06_cat_avg',
 'ps_car_03_cat',
 'ps_car_13',
 'ps_car_09_cat',
 'ps_ind_16_bin',
 'ps_car_04_cat_avg',
 'ps_car_01_cat',
 'ps_ind_06_bin',
 'ps_car_09_cat',
 'ps_reg_01_plus_ps_car_02_cat',
 'ps_ind_07_bin',
 'ps_car_04_cat_avg',
 'ps_car_03_cat_avg']

In [118]:
my_nbe

Model Details
H2ONaiveBayesEstimator :  Naive Bayes
Model Key:  NaiveBayes_model_python_1509299395322_19


ModelMetricsBinomial: naivebayes
** Reported on train data. **

MSE: 0.05151960322064936
RMSE: 0.22697930130443472
LogLoss: 0.2906805048501618
Mean Per-Class Error: 0.4233601112424097
AUC: 0.6050284873575958
Gini: 0.2100569747151917
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06351189495822065: 


,0,1,Error,Rate
0,490821.0,82697.0,0.1442,(82697.0/573518.0)
1,16184.0,5510.0,0.746,(16184.0/21694.0)
Total,507005.0,88207.0,0.1661,(98881.0/595212.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0635119,0.1002721,282.0
max f2,0.0274076,0.1828503,339.0
max f0point5,0.0990246,0.0764898,248.0
max accuracy,0.9998412,0.9598126,0.0
max precision,0.2459148,0.0709916,171.0
max recall,0.0005086,1.0,399.0
max specificity,0.9998412,0.9957996,0.0
max absolute_mcc,0.0446016,0.0595212,308.0
max min_per_class_accuracy,0.0269651,0.5752743,340.0
max mean_per_class_accuracy,0.0274076,0.5766399,339.0


Gains/Lift Table: Avg response rate:  3.64 %



...

metric,threshold,value,idx
max f1,0.0586332,0.0996713,293.0
max f2,0.0273049,0.1827209,341.0
max f0point5,0.0989905,0.0760710,254.0
max accuracy,0.9998247,0.9589373,0.0
max precision,0.2644833,0.0711875,172.0
max recall,0.0004789,1.0,399.0
max specificity,0.9998247,0.9948790,0.0
max absolute_mcc,0.0452690,0.0590587,310.0
max min_per_class_accuracy,0.0267997,0.5744406,342.0
max mean_per_class_accuracy,0.0273049,0.5764350,341.0


Gains/Lift Table: Avg response rate:  3.64 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100015,0.9956771,1.5117153,1.5117153,0.0550983,0.0550983,0.0151194,0.0151194,51.1715283,51.1715283
,2,0.0200013,0.5899067,2.0512998,1.7814849,0.0747648,0.0649307,0.0205126,0.0356320,105.1299768,78.1484864
,3,0.0300011,0.3000152,2.2172476,1.9267310,0.0808132,0.0702246,0.0221720,0.0578040,121.7247615,92.6730980
,4,0.0400009,0.2044547,1.8761326,1.9140819,0.0683804,0.0697635,0.0187609,0.0765649,87.6132597,91.4081915
,5,0.0500007,0.1604350,1.7701104,1.8852886,0.0645161,0.0687141,0.0177007,0.0942657,77.0110362,88.5288572
,6,0.1000013,0.0859680,1.7221120,1.8037003,0.0627667,0.0657404,0.0861068,0.1803725,72.2112006,80.3700289
,7,0.1500003,0.0625036,1.4769358,1.6947812,0.0538306,0.0617706,0.0738453,0.2542178,47.6935833,69.4781244
,8,0.2000010,0.0499549,1.3183191,1.6006649,0.0480495,0.0583403,0.0659168,0.3201346,31.8319148,60.0664929
,9,0.3000007,0.0362973,1.1611545,1.4541623,0.0423212,0.0530006,0.1161151,0.4362497,16.1154450,45.4162257
,10,0.4000003,0.0284461,1.1003079,1.3656990,0.0401035,0.0497763,0.1100304,0.5462801,10.0307929,36.5699046



Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.812067,0.0225744,0.8072854,0.8455915,0.8196508,0.7537044,0.8341027
auc,0.6044838,0.0042072,0.6083382,0.6030792,0.6006957,0.5966402,0.6136655
err,0.1879330,0.0225744,0.1927146,0.1544085,0.1803492,0.2462956,0.1658973
err_count,22386.2,2760.484,22860.0,18355.0,21452.0,29570.0,19694.0
f0point5,0.0725115,0.0027195,0.0742281,0.0733236,0.0703127,0.0666254,0.0780676
f1,0.1003599,0.0026949,0.1033184,0.0992295,0.0971381,0.0959951,0.1061184
f2,0.1635417,0.0050496,0.1699048,0.1534422,0.1570581,0.171671,0.1656324
lift_top_group,1.5074714,0.0883147,1.469277,1.5513306,1.4094505,1.7279187,1.3793802
logloss,0.2894179,0.0062028,0.2864289,0.2902780,0.2944795,0.2748146,0.3010883
max_per_class_error,0.7135248,0.0295228,0.7021036,0.7586536,0.7332408,0.6381655,0.7354605


In [113]:
x=['ps_calc_11',
  'ps_calc_04',
  'ps_ind_04_cat',
  'ps_ind_05_cat',
  'ps_car_06_cat',
  'ps_car_14',
  'ps_car_11_cat',
  'ps_car_13',
  'ps_ind_16_bin',
  'ps_car_09_cat',
  'ps_calc_03',
  'ps_ind_09_bin',
  'ps_car_03_cat',
  'ps_calc_12',
  'ps_ind_03',
  'ps_ind_04_cat',
  'ps_ind_15',
  'ps_reg_01',
  'ps_car_11_cat',
  'ps_calc_10',
  'ps_ind_02_cat',
  'ps_ind_18_bin',
  'ps_car_11_cat',
  'ps_ind_06_bin',
  'ps_car_15',
  'ps_car_15',
  'ps_calc_08',
  'ps_reg_02',
  'ps_ind_10_bin',
  'ps_calc_07',
  'ps_ind_10_bin',
  'ps_car_09_cat',
  'ps_calc_08',
  'ps_calc_12',
  'ps_car_01_cat',
  'ps_ind_11_bin',
  'ps_calc_12',
  'ps_car_07_cat',
  'ps_calc_17_bin',
  'ps_reg_01',
  'ps_ind_16_bin',
  'ps_car_11',
  'ps_ind_17_bin']

In [116]:
x=['ps_car_01_cat',
  'ps_car_06_cat',
  'ps_ind_17_bin',
  'ps_ind_07_bin',
  'ps_reg_02',
  'ps_ind_09_bin',
  'ps_calc_09',
  'ps_ind_16_bin',
  'ps_ind_08_bin',
  'ps_car_01_cat',
  'ps_calc_08',
  'ps_ind_18_bin',
  'ps_car_12',
  'ps_ind_15',
  'ps_ind_07_bin',
  'ps_calc_14',
  'ps_ind_03',
  'ps_calc_03',
  'ps_car_15',
  'ps_calc_07',
  'ps_calc_02',
  'ps_ind_12_bin',
  'ps_ind_12_bin',
  'ps_car_04_cat',
  'ps_ind_15',
  'ps_ind_05_cat',
  'ps_car_07_cat',
  'ps_ind_16_bin',
  'ps_reg_03',
  'ps_calc_11',
  'ps_ind_08_bin',
  'ps_reg_02',
  'ps_calc_04',
  'ps_ind_05_cat',
  'ps_ind_03',
  'ps_ind_18_bin',
  'ps_ind_02_cat',
  'ps_ind_01',
  'ps_car_09_cat',
  'ps_calc_08',
  'ps_ind_03',
  'ps_calc_17_bin',
  'ps_ind_03',
  'ps_ind_05_cat',
  'ps_car_08_cat',
  'ps_calc_02',
  'ps_calc_01']

In [126]:
[(x[0],x[1]) for x in z if x[1]>0.238]

[(['ps_calc_04',
   'ps_ind_16_bin',
   'ps_ind_12_bin',
   'ps_ind_18_bin',
   'ps_reg_03',
   'ps_ind_16_bin',
   'ps_ind_16_bin',
   'ps_ind_03',
   'ps_ind_03',
   'ps_ind_05_cat',
   'ps_car_01_cat',
   'ps_ind_17_bin',
   'ps_car_15',
   'ps_car_09_cat',
   'ps_ind_05_cat',
   'ps_calc_07',
   'ps_ind_15',
   'ps_ind_03',
   'ps_car_12',
   'ps_calc_01',
   'ps_calc_01',
   'ps_calc_02',
   'ps_calc_08',
   'ps_reg_02',
   'ps_ind_12_bin',
   'ps_calc_11',
   'ps_ind_16_bin',
   'ps_car_01_cat',
   'ps_car_07_cat',
   'ps_calc_02',
   'ps_ind_05_cat',
   'ps_calc_14',
   'ps_ind_08_bin',
   'ps_calc_02',
   'ps_ind_08_bin',
   'ps_ind_03',
   'ps_ind_18_bin',
   'ps_reg_02',
   'ps_ind_12_bin',
   'ps_ind_07_bin',
   'ps_calc_03',
   'ps_ind_08_bin',
   'ps_calc_08',
   'ps_ind_16_bin',
   'ps_ind_09_bin',
   'ps_car_12',
   'ps_car_08_cat',
   'ps_ind_01',
   'ps_car_01_cat',
   'ps_ind_18_bin',
   'ps_calc_01',
   'ps_ind_18_bin'],
  0.2386844324622197)]

In [67]:
feature_numbers

array([27])

In [127]:
len(['ps_calc_04',
   'ps_ind_16_bin',
   'ps_ind_12_bin',
   'ps_ind_18_bin',
   'ps_reg_03',
   'ps_ind_16_bin',
   'ps_ind_16_bin',
   'ps_ind_03',
   'ps_ind_03',
   'ps_ind_05_cat',
   'ps_car_01_cat',
   'ps_ind_17_bin',
   'ps_car_15',
   'ps_car_09_cat',
   'ps_ind_05_cat',
   'ps_calc_07',
   'ps_ind_15',
   'ps_ind_03',
   'ps_car_12',
   'ps_calc_01',
   'ps_calc_01',
   'ps_calc_02',
   'ps_calc_08',
   'ps_reg_02',
   'ps_ind_12_bin',
   'ps_calc_11',
   'ps_ind_16_bin',
   'ps_car_01_cat',
   'ps_car_07_cat',
   'ps_calc_02',
   'ps_ind_05_cat',
   'ps_calc_14',
   'ps_ind_08_bin',
   'ps_calc_02',
   'ps_ind_08_bin',
   'ps_ind_03',
   'ps_ind_18_bin',
   'ps_reg_02',
   'ps_ind_12_bin',
   'ps_ind_07_bin',
   'ps_calc_03',
   'ps_ind_08_bin',
   'ps_calc_08',
   'ps_ind_16_bin',
   'ps_ind_09_bin',
   'ps_car_12',
   'ps_car_08_cat',
   'ps_ind_01',
   'ps_car_01_cat',
   'ps_ind_18_bin',
   'ps_calc_01',
   'ps_ind_18_bin'])

52